# Faiss

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [22]:
embeddings = OllamaEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db

In [ ]:
# Querying

query = "How does the user describes the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## As a retriever

In [ ]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## Similarity Search with Score

In [21]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='f5a40891-d46a-4362-9ffd-694424363a3e', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(14021.1)),
 (Document(id='b511e12e-5b1b-4462-a35a-498d7899132e', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say 

In [23]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.5788940787315369,
 -0.843529224395752,
 2.3203823566436768,
 0.06289005279541016,
 -1.6780787706375122,
 1.6146113872528076,
 1.5464224815368652,
 0.03575922176241875,
 -0.10281692445278168,
 -1.4327571392059326,
 1.3943135738372803,
 -1.303863286972046,
 -0.5153600573539734,
 1.6860761642456055,
 -0.350446492433548,
 -2.293323278427124,
 0.9574511051177979,
 0.4804539978504181,
 1.2795219421386719,
 -1.9848836660385132,
 -1.0497334003448486,
 -0.6930357813835144,
 1.3578426837921143,
 -1.8518986701965332,
 0.6649336218833923,
 -0.8237828612327576,
 0.5543959736824036,
 -0.6259080767631531,
 -0.4652327299118042,
 -1.3542529344558716,
 3.0784287452697754,
 -1.8657792806625366,
 -2.5797483921051025,
 4.1948018074035645,
 1.9260443449020386,
 -4.013686180114746,
 -0.32904037833213806,
 0.9753623008728027,
 -0.6533704996109009,
 -0.9913303256034851,
 -1.334595799446106,
 -3.582670211791992,
 1.2753921747207642,
 -0.7160388827323914,
 1.388346791267395,
 -0.5715983510017395,
 0.104426264

In [24]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='eda5588e-7a4b-4c11-a244-fcc539062b88', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='b1e1cbd1-dcb8-4c4c-ab01-1c5243fe5464', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends

In [31]:
# Saving
db.save_local("faiss_index")

In [32]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)
docs

[Document(id='eda5588e-7a4b-4c11-a244-fcc539062b88', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='b1e1cbd1-dcb8-4c4c-ab01-1c5243fe5464', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends